#### Module Initialization

In [23]:
## -- Module WAXS Classes -- ##
from WAXSTransform import WAXSTransform
from WAXSReduce import WAXSReduce
from WAXSReduce import ImageInterpolator
from WAXSReduce import Integration1D
from WAXSReduce import Azimuth1D
from WAXSReduce import ImageTransform
from WAXSSearch import WAXSSearch
from WAXSDiffSim import WAXSDiffSim

# - Import Relevant Modules"
import xarray as xr
import numpy as np
import pathlib, os
from pathlib import Path
from typing import Optional
import matplotlib.pyplot as plt
from math import cos, sin, radians

# - pymatgen: https://pymatgen.org/
# from pymatgen import Structure, Lattice
from pymatgen.core import Structure, Lattice, Molecule, IMolecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter

#### Path Definitions

In [24]:
# -- Base Path Definitions -- #
basePath = pathlib.Path('/Users/keithwhite/github_repositories/pyWAXS/examples_local/simulations')

# -- Project Path Definitions -- #
cifPath = basePath.joinpath('cif')
waxsPath = basePath.joinpath('waxs')


#### Example: Single-Atom Basis

In [25]:
# from pymatgen.core import Structure, Lattice
# from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Create a cubic lattice
lattice = Lattice.cubic(3.567)

# Specify species and coordinates
species = ['C']
coords = [[0, 0, 0]]  # Wyckoff position 'a' for Fd-3m

# Create a Structure object
structure = Structure.from_spacegroup("Fd-3m", lattice, species, coords)

# Analyze the structure with the spacegroup analyzer
analyzer = SpacegroupAnalyzer(structure)

# Get the primitive standardized structure
prim_structure = analyzer.get_primitive_standard_structure()

# Print information
print("Spacegroup Symbol:", analyzer.get_space_group_symbol())
print("International Number:", analyzer.get_space_group_number())
print("Crystal System:", analyzer.get_crystal_system())

# Write CIF file to cifPath
cif_file_path = cifPath.joinpath("diamond.cif")
writer = CifWriter(prim_structure)
writer.write_file(cif_file_path)


Spacegroup Symbol: Fd-3m
International Number: 227
Crystal System: cubic


#### Example: Multi-Atom Basis

In [26]:
# from pymatgen.core import Structure, Lattice
# from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
# from pymatgen.io.cif import CifWriter

# Create a cubic lattice
lattice = Lattice.cubic(5.692)

# Specify species and coordinates
species = ['Na', 'Cl']
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]  # Wyckoff positions 'a' and 'b' for Fm-3m

# Create a Structure object
structure = Structure.from_spacegroup("Fm-3m", lattice, species, coords)

# Analyze the structure with the spacegroup analyzer
analyzer = SpacegroupAnalyzer(structure)

# Get the primitive standardized structure
prim_structure = analyzer.get_primitive_standard_structure()

# Print information
print("Spacegroup Symbol:", analyzer.get_space_group_symbol())
print("International Number:", analyzer.get_space_group_number())
print("Crystal System:", analyzer.get_crystal_system())

# Write CIF file to cifPath
cif_file_path = cifPath.joinpath("NaCl.cif")
writer = CifWriter(prim_structure)
writer.write_file(cif_file_path)

Spacegroup Symbol: Fm-3m
International Number: 225
Crystal System: cubic


#### Example: Generate CIF with Fractional Occupancy

In [5]:
# Define lattice and species
lattice = Lattice.cubic(5.692)
species = [{"Na": 0.5, "K": 0.5}, {"Cl": 1.0}]
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]

# Create the Structure object
structure = Structure(lattice, species, coords)

# Write to CIF file
cif_file_path = cifPath.joinpath("Na_K_Cl_mixed.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### Molecule Definitions

In [27]:
def water_molecule():
    angle = 104.5  # H-O-H angle in degrees
    angle_rad = radians(angle)
    bond_length = 0.9572  # O-H bond length in Angstrom

    # Coordinates with O at the center of the unit cell in fractional coordinates
    O = [0.5, 0.5, 0.5]
    H1 = [0.5 + bond_length * cos(angle_rad / 2) / 20, 
          0.5 + bond_length * sin(angle_rad / 2) / 20, 
          0.5]
    H2 = [0.5 - bond_length * cos(angle_rad / 2) / 20, 
          0.5 + bond_length * sin(angle_rad / 2) / 20, 
          0.5]
    
    return Molecule(["O", "H", "H"], [O, H1, H2])


def methane():
    coords = [[0, 0, 0], [0, 0.629118, -1.04704], [0, -1.25765, 0], [1.09041, 0, 0.523519], [-1.09041, 0, 0.523519]]
    return Molecule(["C", "H", "H", "H", "H"], coords)

def methylammonium():
    coords = [
        [0.000, 0.000, 0.000],  # N
        [0.000, 0.000, 1.000],  # H
        [0.866, 0.000, -0.500],  # H
        [-0.433, 0.750, -0.500],  # H
        [-0.433, -0.750, -0.500],  # H
        [0.000, 0.000, -1.500],  # C
        [0.000, 0.000, -2.500]  # H (methyl)
    ]
    return Molecule(["N", "H", "H", "H", "H", "C", "H"], coords)

def dimethylformamide():
    coords = [
        # O, C, N
        [0.000, 0.000, 0.000],
        [0.000, 0.000, 1.200],
        [0.000, 0.000, 2.400],
        # H atoms on N
        [0.866, 0.000, 2.900],
        [-0.866, 0.000, 2.900],
        # H atoms on C (methyl)
        [0.866, 0.000, -0.500],
        [-0.433, 0.750, -0.500],
        [-0.433, -0.750, -0.500]
    ]
    return Molecule(["O", "C", "N", "H", "H", "H", "H", "H"], coords)

def dimethylsulfoxide():
    coords = [
        # S, O
        [0.000, 0.000, 0.000],
        [0.000, 0.000, 1.600],
        # C atoms
        [0.866, 0.000, -0.500],
        [-0.866, 0.000, -0.500],
        # H atoms on C (methyl)
        [1.732, 0.000, 0.000],
        [0.433, 0.750, -1.000],
        [0.433, -0.750, -1.000],
        [-1.732, 0.000, 0.000],
        [-0.433, 0.750, -1.000],
        [-0.433, -0.750, -1.000]
    ]
    return Molecule(["S", "O", "C", "C", "H", "H", "H", "H", "H", "H"], coords)

def formamidinium():
    coords = [
        [0.000, 0.000, 0.000],  # C
        [0.866, 0.000, 0.500],  # H
        [-0.433, 0.750, 0.500],  # H
        [-0.433, -0.750, 0.500],  # H
        [0.000, 0.000, 1.500],  # N
        [0.866, 0.000, 2.000],  # H
        [-0.866, 0.000, 2.000],  # H
    ]
    return Molecule(["C", "H", "H", "H", "N", "H", "H"], coords)


#### Generate CIFs of Molecules

In [28]:
# Create a large cubic lattice
lattice = Lattice.cubic(20.0)  # 20 Angstroms on a side

# Use the corrected water molecule
water = water_molecule()
species = water.species
coords = water.cart_coords

# Create Structure
structure = Structure(lattice, species, coords)

# Save to CIF
cif_file_path = cifPath.joinpath("corrected_centered_water.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### Example: Assigning Charge to Atomic Species

In [12]:
# Cubic lattice
lattice = Lattice.cubic(5.64)  # Lattice parameter for NaCl

# Species and Coordinates with charges
species = ["Na1+", "Cl1-"]
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]

# Create Structure
structure = Structure(lattice, species, coords)

# Save to CIF
cif_file_path = cifPath.joinpath("NaCl_charged.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)